In [1]:
import polars as pl
from pathlib import Path
import h3ronpy.polars

In [39]:
csvs = list(Path("../data/raw/ENTREGA UNO MUESTRAS HEXA CSV 18072024").glob("*.CSV"))

In [42]:
dfs = [pl.read_csv(f, separator=";", decimal_comma=True) for f in csvs]
df = pl.concat(dfs, how="align", rechunk=True)
df.head()

GRID_ID,FRECF,AMIN,AMAX,AMEAN,TREEPERCT,PMIN,PMAX,PMEAN
str,i64,f64,f64,f64,f64,f64,f64,f64
"""865f00007ffffff""",null,114.678246,209.731842,149.513126,100.0,0.058348,7.531753,1.69093
"""865f0000fffffff""",null,127.660339,705.040772,245.461013,99.985832,0.148311,31.043549,6.346733
"""865f00017ffffff""",null,117.937508,175.799759,145.636984,100.0,0.028819,2.731335,1.063382
"""865f0001fffffff""",null,123.765045,193.208282,156.474098,100.0,0.047981,4.67722,1.557258
"""865f00027ffffff""",null,111.118088,277.398895,146.417323,100.0,0.144035,12.342467,2.193349


In [43]:
df = df.with_columns(pl.col("GRID_ID").h3.cells_parse())
df = df.drop("GRID_ID")

In [44]:
df.select(pl.col("cell").h3.cells_resolution()).unique()

resolution
u8
6


In [45]:
CELLS_RES = 6
OVERVIEW_LEVEL = CELLS_RES - 5

df = df.with_columns(
    pl.col("cell").h3.change_resolution(OVERVIEW_LEVEL).h3.cells_to_string().alias("tile_id"),  # type: ignore[attr-defined]
    pl.col("cell").h3.cells_to_string()
)
partition_dfs = df.partition_by(["tile_id"], as_dict=True, include_key=False)

## Write tiles to IPC files

In [48]:
seen_tiles = set()
n_cells = 0

for tile_group, tile_df in partition_dfs.items():
    if tile_df.shape[0] == 0:  # todo: skip empty tiles ?
        continue
    tile_id = tile_group[0]
    filename = Path("../data/processed/grid/1") / (tile_id + ".arrow")
    if tile_id in seen_tiles:
        tile_df = pl.concat(
            [pl.read_ipc(filename), tile_df], how="vertical_relaxed"
        ).unique(subset=["cell"])
        tile_df.write_parquet(filename)
        n_cells += len(tile_df)
    else:
        seen_tiles.add(tile_id)
        tile_df.write_ipc(filename)
        n_cells += len(tile_df)

## Make the metadata

In [ ]:
df.select(pl.all().min())

In [ ]:
df.select(pl.all().max())

In [ ]:
df.columns

In [ ]:
df.dtypes